# Debugging a naming issue

Took me almost an hour of headscratching to figure out why the hell Türkiye wasn't matching across the datasets.. 

In [165]:
# In[5]:
import pandas as pd
import re

# --- CONFIGURATION ---
NVIVO_FILE = 'matrix_coding_query.xlsx'
LLM_FILE = 'LLM_output_test.csv'

print("--- Forensic Analysis on 'Türkiye' ---")

try:
    # --- LOAD DATA ---
    df_llm = pd.read_csv(LLM_FILE)
    df_nvivo = pd.read_excel(NVIVO_FILE)

    # --- Isolate the LLM string ---
    llm_turkiye_str = None
    llm_row = df_llm[df_llm['Country'].str.contains('rkiye', na=False)] # Use contains to be safe
    if not llm_row.empty:
        llm_turkiye_str = llm_row['Country'].iloc[0]
        print("\nFound a 'Türkiye'-like string in LLM DataFrame.")
    else:
        print("\n❌ Could not find 'Türkiye' in LLM DataFrame.")

    # --- Isolate the NVIVO string ---
    nvivo_turkiye_str_raw = None
    nvivo_row = df_nvivo[df_nvivo['Country'].str.contains('rkiye', na=False)]
    if not nvivo_row.empty:
        nvivo_turkiye_str_raw = nvivo_row['Country'].iloc[0]
        print("Found a 'Türkiye'-like string in NVIVO DataFrame.")
    else:
        print("\n❌ Could not find 'Türkiye' in NVIVO DataFrame.")

    # --- Compare the strings if both were found ---
    if llm_turkiye_str and nvivo_turkiye_str_raw:
        # Clean the NVIVO string by removing the prefix
        nvivo_turkiye_clean_str = re.sub(r'^\d+\s*:\s*', '', nvivo_turkiye_str_raw).strip()
        
        print("\n--- String Comparison ---")
        print(f"LLM String:           '{llm_turkiye_str}' (Length: {len(llm_turkiye_str)})")
        print(f"NVIVO String (Clean): '{nvivo_turkiye_clean_str}' (Length: {len(nvivo_turkiye_clean_str)})")
        
        print(f"\nDirect comparison (==): {llm_turkiye_str == nvivo_turkiye_clean_str}")
        
        print("\n--- Byte-Level Representation (UTF-8) ---")
        print(f"LLM Bytes:            {llm_turkiye_str.encode('utf-8')}")
        print(f"NVIVO Bytes:          {nvivo_turkiye_clean_str.encode('utf-8')}")
        
        if llm_turkiye_str.encode('utf-8') == nvivo_turkiye_clean_str.encode('utf-8'):
            print("\n✅ The byte representations are IDENTICAL. The problem is elsewhere.")
        else:
            print("\n❌ BINGO! The byte representations are DIFFERENT. This is the root cause.")
    else:
        print("\nCould not find 'Türkiye' in both DataFrames to perform a comparison.")

except FileNotFoundError as e:
    print(f"❌ ERROR: Could not find input file. Make sure filenames are correct.")
    print(e)

--- Forensic Analysis on 'Türkiye' ---

Found a 'Türkiye'-like string in LLM DataFrame.
Found a 'Türkiye'-like string in NVIVO DataFrame.

--- String Comparison ---
LLM String:           'Türkiye' (Length: 7)
NVIVO String (Clean): 'Türkiye' (Length: 8)

Direct comparison (==): False

--- Byte-Level Representation (UTF-8) ---
LLM Bytes:            b'T\xc3\xbcrkiye'
NVIVO Bytes:          b'Tu\xcc\x88rkiye'

❌ BINGO! The byte representations are DIFFERENT. This is the root cause.


# Manual review of randomly sampled output data

I then took a few randomly selected countries to manually review to ensure the JSON parsing had been done correctly.

In [166]:
import pandas as pd

In [167]:
df_LLM = pd.read_csv("llm_output_test.csv")

In [168]:
# These settings ensure you can see all the columns in the output without anything being hidden.
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) 
pd.set_option('display.max_rows', None)
# --- Main Inspection Code ---

# 1. Load the CSV file generated by your previous script.
#    Make sure the filename 'llm_output.csv' matches what you've generated.
df_llm = pd.read_csv("llm_output_test.csv")

print(f"✅ Successfully loaded 'llm_output.csv'. Shape: {df_llm.shape}")
print("-" * 50)

    # 2. Select 3 random rows from the DataFrame.
    #    Using a 'random_state' makes the selection repeatable if you run the code again.
random_sample = df_llm.sample(n=3, random_state=42)

    # 3. Transpose (.T) the sample for easy vertical reading and print it.
print("Inspecting 3 random rows (transposed for readability):")
    # Using display() is often better for dataframes in environments like Jupyter
display(random_sample.T)

✅ Successfully loaded 'llm_output.csv'. Shape: (69, 87)
--------------------------------------------------
Inspecting 3 random rows (transposed for readability):


,22,0,47
Country,GEORGIA,AOSIS,Peru
A : C1 Objectives - end plastic pollution,0,0,0
B : Mentioned with time frame,0,0,0
"C : Mentioned, no time frame",1,1,1
D : Not mentioned,0,0,0
E : C2 Objectives - reduce production of plastics,0,0,0
F : Mentioned with specification,0,0,0
"G : Mentioned, no specification",1,1,1
H : Not mentioned,0,0,0
I : C3 Objectives - benefits of plastics,0,0,0


They're correct (feel free to check). So now we can continue on to the alignment process and ensure we are comparing correctly.
We need to make sure the columns are placed in the right order, and then ensure that the countries are in the right order row-wise.

In [169]:
df_nvivo = pd.read_excel("matrix_coding_query.xlsx") # Our handy-dandy golden standard dataset

In [170]:
df_nvivo.head(5)

,Country,A : C1 Objectives - end plastic pollution,B : Mentioned with time frame,"C : Mentioned, no time frame",D : Not mentioned,E : C2 Objectives - reduce production of plastics,F : Mentioned with specification,"G : Mentioned, no specification",H : Not mentioned,I : C3 Objectives - benefits of plastics,J : Mentioned,K : Not mentioned,L : C4 Objectives - protect human health,M : Mentioned,N : Not mentioned,O : C5 Objectives - protect biodiversity and (marine) environment,P : Mentioned,Q : Not mentioned,R : C6 Objectives - addressing the full life cycle of plastics,S : Mentioned,T : Not mentioned,U : Partial mention,V : C7 Objectives - other objectives,W : Circular economy,X : Climate change,Y : ESM,Z : Mentioned,AA : Not mentioned,AB : Sustainable production,AC : C8 Value chain,AD : 1. Upstream,"AE : 1. Feedstock and raw materials (fossil, biobased and recycled)",AF : 2. Production and processing of monomers and polymers,AG : 2. Midstream,AH : 1. Design phase and innovation of plastic products,AI : 2. Production of plastic products,"AJ : 1. Harmful chemical additives of concern for the environment, and human health and safety (e.g., toxic to human health)",AK : 2. Intentionally added microplastics,AL : 3. Distribution,AM : 4. Plastics use and consumption,"AN : Avoidable plastic - unnecessary, short lived and single use",AO : 3. Downstream,"AP : 1. Collection, sorting and waste management","AQ : 2. Reuse, repair and recycling",AR : 3. Legacy plastic,AS : 4. Cross value chain,"AT : 1. Release and emission to water, soil and air",AU : 2. Microplastic leakage,AV : C9 Type of measure,AW : Instrument,AX : Economic,AY : Deposit systems,AZ : Penalty,BA : Public procurement,BB : R&D funding,BC : Subsidy,BD : Tax incentive,BE : Trading system,BF : Regulatory,BG : Ban,BH : EPR,BI : Legal recognition - Just transition,BJ : Mandatory action plan,BK : Mandatory certification,BL : Mandatory infrastructure,BM : Mandatory labelling,BN : Mandatory reports,BO : Moratorium,BP : Performance standard,BQ : Requirements & surveillance in trade systems,BR : Quota,BS : Soft,"BT : Assessment, monitoring and evaluation",BU : Voluntary reports,BV : Education programs and awareness raising,BW : Expert group,BX : Harmonization,BY : Information and guidance,BZ : Knowledge sharing,CA : Capacity building,CB : Data registry,CC : Joint research projects,CD : Technology transfer,CE : Promotion of research & innovation,CF : Voluntary certification,CG : Voluntary labelling,CH : Target
0,1 : African Group,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,1,4,2,3,1,0,0,1,1,7,6,4,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,2,0,0,1,1,0,2,1,0,0,2,0,2,1,1,0,0,0,0,0,1,0,1,0,1
1,2 : AOSIS,0,0,2,0,0,0,0,1,0,0,2,0,1,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,7,6,1,1,1,0,0,1,10,2,8,0,6,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,2,0,2,1,0,0,2,0,1,2,0,0,0,2,0,0,1,1,0,1,1
2,3 : Argentina,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,3,1,0,1,0,2,0,0,7,4,5,0,11,1,0,0,0,0,0,1,0,1,0,0,0,0,1,3,0,1,0,0,0,1,0,1,2,0,0,2,0,3,3,0,0,0,2,1,0,1,1,0,0,0
3,4 : Armenia,0,0,1,0,0,0,1,0,0,0,2,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0,2,2,1,0,0,0,1,1,6,2,5,0,4,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,1,0,0,0,0
4,5 : Australia,0,0,1,0,0,0,0,1,0,0,1,0,2,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,4,3,1,10,8,3,3,0,0,0,2,9,1,9,1,10,4,1,0,0,0,0,1,0,0,0,0,0,0,2,0,0,1,0,0,2,4,0,1,2,0,0,2,1,0,1,2,2,0,2,1,0,1,1,0,0,0


In [171]:


# I remove the numerical prefixes. ie "1 : African Group" becomes "African Group"
df_nvivo['Country'] = df_nvivo['Country'].str.replace(r'^\d+\s*:\s*', '', regex=True).str.strip()

# Display the first few rows to verify the result
display(df_nvivo[['Country']].head())

,Country
0,African Group
1,AOSIS
2,Argentina
3,Armenia
4,Australia


Quick manual look at the columns.

In [172]:
df_nvivo.columns.to_list()

['Country',
 'A : C1 Objectives - end plastic pollution',
 'B : Mentioned with time frame',
 'C : Mentioned, no time frame',
 'D : Not mentioned',
 'E : C2 Objectives - reduce production of plastics',
 'F : Mentioned with specification',
 'G : Mentioned, no specification',
 'H : Not mentioned',
 'I : C3 Objectives - benefits of plastics',
 'J : Mentioned',
 'K : Not mentioned',
 'L : C4 Objectives - protect human health',
 'M : Mentioned',
 'N : Not mentioned',
 'O : C5 Objectives - protect biodiversity and (marine) environment',
 'P : Mentioned',
 'Q : Not mentioned',
 'R : C6 Objectives - addressing the full life cycle of plastics',
 'S : Mentioned',
 'T : Not mentioned',
 'U : Partial mention',
 'V : C7 Objectives - other objectives',
 'W : Circular economy',
 'X : Climate change',
 'Y : ESM',
 'Z : Mentioned',
 'AA : Not mentioned',
 'AB : Sustainable production',
 'AC : C8 Value chain',
 'AD : 1. Upstream',
 'AE : 1. Feedstock and raw materials (fossil, biobased and recycled)',
 '

In [173]:
df_nvivo.Country.to_list()

['African Group',
 'AOSIS',
 'Argentina',
 'Armenia',
 'Australia',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Bosnia and Herzegovina',
 'Brazil',
 'Burkina Faso',
 'Cambodia',
 'Canada',
 'China',
 'Colombia',
 'Cook Islands',
 'Ecuador',
 'Egypt',
 'EU',
 'Gabon',
 'Georgia',
 'Ghana',
 'GRULAC',
 'Guinea',
 'HAC',
 'Iceland',
 'Indonesia',
 'Iran',
 'Japan',
 'Kenya',
 'Korea, Republic of',
 'Kuwait',
 'Libya',
 'Malaysia',
 'Mauritius',
 'Micronesia',
 'Moldova',
 'Monaco',
 'Morocco',
 'Nepal',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Oman',
 'Palau',
 'Palestine',
 'Papa New Guinea',
 'Peru',
 'Philippines',
 'Qatar',
 'Russia',
 'Rwanda',
 'Saudi Arabia',
 'Sierra Leone',
 'Singapore',
 'Sri Lanka',
 'Switzerland',
 'Syria',
 'Tanzania',
 'Thailand',
 'Tonga',
 'Tunisia',
 'Türkiye',
 'Uganda',
 'United Kingdom',
 'Uruguay',
 'USA',
 'Yemen']

Pythonic check with equals(), and it looks like the columns match.

In [175]:
# Column name alignment check

arrayLLM = df_LLM.columns
arrayNvivo = df_nvivo.columns

arrayLLM.equals(arrayNvivo)

True

In [176]:
# Country name alignment check
countryArrayLLM = df_LLM.Country
countryArrayNvivo = df_nvivo.Country

countryArrayLLM.equals(countryArrayNvivo)

False

In [177]:
list(df_nvivo.Country)

['African Group',
 'AOSIS',
 'Argentina',
 'Armenia',
 'Australia',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Bosnia and Herzegovina',
 'Brazil',
 'Burkina Faso',
 'Cambodia',
 'Canada',
 'China',
 'Colombia',
 'Cook Islands',
 'Ecuador',
 'Egypt',
 'EU',
 'Gabon',
 'Georgia',
 'Ghana',
 'GRULAC',
 'Guinea',
 'HAC',
 'Iceland',
 'Indonesia',
 'Iran',
 'Japan',
 'Kenya',
 'Korea, Republic of',
 'Kuwait',
 'Libya',
 'Malaysia',
 'Mauritius',
 'Micronesia',
 'Moldova',
 'Monaco',
 'Morocco',
 'Nepal',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Oman',
 'Palau',
 'Palestine',
 'Papa New Guinea',
 'Peru',
 'Philippines',
 'Qatar',
 'Russia',
 'Rwanda',
 'Saudi Arabia',
 'Sierra Leone',
 'Singapore',
 'Sri Lanka',
 'Switzerland',
 'Syria',
 'Tanzania',
 'Thailand',
 'Tonga',
 'Tunisia',
 'Türkiye',
 'Uganda',
 'United Kingdom',
 'Uruguay',
 'USA',
 'Yemen']

In [178]:
list(df_LLM.Country)

['AOSIS',
 'Africa Group',
 'ARGENTINA',
 'Republic of Armenia',
 'Australia',
 'Azerbaijan',
 'KINGDOM OF BAHRAIN',
 'Bangladesh',
 'Bosnia and Herzegovina',
 'Brazil',
 'Burkina Faso',
 'Cambodia',
 'Canada',
 "People's Republic of China",
 'COLOMBIA',
 'COOK ISLANDS',
 'European Union and its 27 Member States',
 'ECUADOR',
 'The Arab Republic of Egypt',
 'REPUBLIC OF EQUATORIAL GUINEA',
 'Costa Rica on behalf of the Group of Latin America and the Caribbean Countries (GRULAC)',
 'GABON',
 'GEORGIA',
 'Ghana',
 'Republic of Guinea',
 'Norway & Rwanda as co-chairs of the High Ambition Coalition to End Plastic Pollution',
 'Iceland',
 'Indonesia',
 'Islamic Republic of Iran',
 'Japan',
 'Kenya',
 'State of Kuwait',
 'Libya',
 'Malaysia',
 'Republic of Mauritius',
 'Federated States of Micronesia',
 'Republic of Moldova',
 'PRINCIPALITY OF MONACO',
 'Morocco',
 'Nepal',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Sultanate of Oman',
 'Palau',
 'State of PALESTINE',
 'Papua New Guinea',
 'Pe

Alright, I think we have all the contextual clues we need.

# IRR Analysis begins here

## First we need to align the data frames to ensure the analysis is done correctly. 

The columns we have already checked the order matching, but we need to align the countries as well

In [179]:

# --- CONFIGURATION ---
# Make sure the CSV filenames match these
NVIVO_FILE = 'matrix_coding_query.xlsx'
LLM_FILE = 'llm_output_test.csv'
ALIGNED_LLM_OUTPUT_FILE = 'llm_output_aligned.csv'

# --- LOAD DATA ---
try:
    df_llm = pd.read_csv(LLM_FILE)
    df_nvivo = pd.read_excel(NVIVO_FILE)
    # I remove the numerical prefixes. ie "1 : African Group" becomes "African Group"
    df_nvivo['Country'] = df_nvivo['Country'].str.replace(r'^\d+\s*:\s*', '', regex=True).str.strip()
    print("✅ DataFrames loaded successfully.")
    print("\nLLM DataFrame head:")
    display(df_llm.head(3))
    print("\nNVIVO DataFrame head:")
    display(df_nvivo.head(3))
except FileNotFoundError as e:
    print(f"❌ ERROR: Could not find input file. Make sure '{NVIVO_FILE}' and '{LLM_FILE}' are in the correct directory.")
    print(e)

✅ DataFrames loaded successfully.

LLM DataFrame head:


,Country,A : C1 Objectives - end plastic pollution,B : Mentioned with time frame,"C : Mentioned, no time frame",D : Not mentioned,E : C2 Objectives - reduce production of plastics,F : Mentioned with specification,"G : Mentioned, no specification",H : Not mentioned,I : C3 Objectives - benefits of plastics,J : Mentioned,K : Not mentioned,L : C4 Objectives - protect human health,M : Mentioned,N : Not mentioned,O : C5 Objectives - protect biodiversity and (marine) environment,P : Mentioned,Q : Not mentioned,R : C6 Objectives - addressing the full life cycle of plastics,S : Mentioned,T : Not mentioned,U : Partial mention,V : C7 Objectives - other objectives,W : Circular economy,X : Climate change,Y : ESM,Z : Mentioned,AA : Not mentioned,AB : Sustainable production,AC : C8 Value chain,AD : 1. Upstream,"AE : 1. Feedstock and raw materials (fossil, biobased and recycled)",AF : 2. Production and processing of monomers and polymers,AG : 2. Midstream,AH : 1. Design phase and innovation of plastic products,AI : 2. Production of plastic products,"AJ : 1. Harmful chemical additives of concern for the environment, and human health and safety (e.g., toxic to human health)",AK : 2. Intentionally added microplastics,AL : 3. Distribution,AM : 4. Plastics use and consumption,"AN : Avoidable plastic - unnecessary, short lived and single use",AO : 3. Downstream,"AP : 1. Collection, sorting and waste management","AQ : 2. Reuse, repair and recycling",AR : 3. Legacy plastic,AS : 4. Cross value chain,"AT : 1. Release and emission to water, soil and air",AU : 2. Microplastic leakage,AV : C9 Type of measure,AW : Instrument,AX : Economic,AY : Deposit systems,AZ : Penalty,BA : Public procurement,BB : R&D funding,BC : Subsidy,BD : Tax incentive,BE : Trading system,BF : Regulatory,BG : Ban,BH : EPR,BI : Legal recognition - Just transition,BJ : Mandatory action plan,BK : Mandatory certification,BL : Mandatory infrastructure,BM : Mandatory labelling,BN : Mandatory reports,BO : Moratorium,BP : Performance standard,BQ : Requirements & surveillance in trade systems,BR : Quota,BS : Soft,"BT : Assessment, monitoring and evaluation",BU : Voluntary reports,BV : Education programs and awareness raising,BW : Expert group,BX : Harmonization,BY : Information and guidance,BZ : Knowledge sharing,CA : Capacity building,CB : Data registry,CC : Joint research projects,CD : Technology transfer,CE : Promotion of research & innovation,CF : Voluntary certification,CG : Voluntary labelling,CH : Target
0,AOSIS,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,0
1,Africa Group,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,0,0,1,1,0,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,0
2,ARGENTINA,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,0,0,0,1,0,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,1,0,0,0



NVIVO DataFrame head:


,Country,A : C1 Objectives - end plastic pollution,B : Mentioned with time frame,"C : Mentioned, no time frame",D : Not mentioned,E : C2 Objectives - reduce production of plastics,F : Mentioned with specification,"G : Mentioned, no specification",H : Not mentioned,I : C3 Objectives - benefits of plastics,J : Mentioned,K : Not mentioned,L : C4 Objectives - protect human health,M : Mentioned,N : Not mentioned,O : C5 Objectives - protect biodiversity and (marine) environment,P : Mentioned,Q : Not mentioned,R : C6 Objectives - addressing the full life cycle of plastics,S : Mentioned,T : Not mentioned,U : Partial mention,V : C7 Objectives - other objectives,W : Circular economy,X : Climate change,Y : ESM,Z : Mentioned,AA : Not mentioned,AB : Sustainable production,AC : C8 Value chain,AD : 1. Upstream,"AE : 1. Feedstock and raw materials (fossil, biobased and recycled)",AF : 2. Production and processing of monomers and polymers,AG : 2. Midstream,AH : 1. Design phase and innovation of plastic products,AI : 2. Production of plastic products,"AJ : 1. Harmful chemical additives of concern for the environment, and human health and safety (e.g., toxic to human health)",AK : 2. Intentionally added microplastics,AL : 3. Distribution,AM : 4. Plastics use and consumption,"AN : Avoidable plastic - unnecessary, short lived and single use",AO : 3. Downstream,"AP : 1. Collection, sorting and waste management","AQ : 2. Reuse, repair and recycling",AR : 3. Legacy plastic,AS : 4. Cross value chain,"AT : 1. Release and emission to water, soil and air",AU : 2. Microplastic leakage,AV : C9 Type of measure,AW : Instrument,AX : Economic,AY : Deposit systems,AZ : Penalty,BA : Public procurement,BB : R&D funding,BC : Subsidy,BD : Tax incentive,BE : Trading system,BF : Regulatory,BG : Ban,BH : EPR,BI : Legal recognition - Just transition,BJ : Mandatory action plan,BK : Mandatory certification,BL : Mandatory infrastructure,BM : Mandatory labelling,BN : Mandatory reports,BO : Moratorium,BP : Performance standard,BQ : Requirements & surveillance in trade systems,BR : Quota,BS : Soft,"BT : Assessment, monitoring and evaluation",BU : Voluntary reports,BV : Education programs and awareness raising,BW : Expert group,BX : Harmonization,BY : Information and guidance,BZ : Knowledge sharing,CA : Capacity building,CB : Data registry,CC : Joint research projects,CD : Technology transfer,CE : Promotion of research & innovation,CF : Voluntary certification,CG : Voluntary labelling,CH : Target
0,African Group,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,1,4,2,3,1,0,0,1,1,7,6,4,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,2,0,0,1,1,0,2,1,0,0,2,0,2,1,1,0,0,0,0,0,1,0,1,0,1
1,AOSIS,0,0,2,0,0,0,0,1,0,0,2,0,1,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,7,6,1,1,1,0,0,1,10,2,8,0,6,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,2,0,2,1,0,0,2,0,1,2,0,0,0,2,0,0,1,1,0,1,1
2,Argentina,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,3,1,0,1,0,2,0,0,7,4,5,0,11,1,0,0,0,0,0,1,0,1,0,0,0,0,1,3,0,1,0,0,0,1,0,1,2,0,0,2,0,3,3,0,0,0,2,1,0,1,1,0,0,0


In [ ]:
# This dictionary maps the messy LLM names (keys) to the clean NVIVO names (values).
COUNTRY_MAPPING = {
    # Messy LLM Name (Key) : Clean NVIVO Name (Value)
    "Africa Group": "African Group",
    "AOSIS": "AOSIS",
    "ARGENTINA": "Argentina",
    "Republic of Armenia": "Armenia",
    "Australia": "Australia",
    "Azerbaijan": "Azerbaijan",
    "KINGDOM OF BAHRAIN": "Bahrain",
    "Bangladesh": "Bangladesh",
    "Bosnia and Herzegovina": "Bosnia and Herzegovina",
    "Brazil": "Brazil",
    "Burkina Faso": "Burkina Faso",
    "Cambodia": "Cambodia",
    "Canada": "Canada",
    "People's Republic of China": "China",
    "COLOMBIA": "Colombia",
    "COOK ISLANDS": "Cook Islands",
    "ECUADOR": "Ecuador",
    "The Arab Republic of Egypt": "Egypt",
    "European Union and its 27 Member States": "EU",
    "GABON": "Gabon",
    "GEORGIA": "Georgia",
    "Ghana": "Ghana",
    "Costa Rica on behalf of the Group of Latin America and the Caribbean Countries (GRULAC)": "GRULAC",
    "Republic of Guinea": "Guinea",
    "Norway & Rwanda as co-chairs of the High Ambition Coalition to End Plastic Pollution": "HAC",
    "Iceland": "Iceland",
    "Indonesia": "Indonesia",
    "Islamic Republic of Iran": "Iran",
    "Japan": "Japan",
    "Kenya": "Kenya",
    "Republic of Korea": "Korea, Republic of",
    "State of Kuwait": "Kuwait",
    "Libya": "Libya",
    "Malaysia": "Malaysia",
    "Republic of Mauritius": "Mauritius",
    "Federated States of Micronesia": "Micronesia",
    "Republic of Moldova": "Moldova",
    "PRINCIPALITY OF MONACO": "Monaco",
    "Morocco": "Morocco",
    "Nepal": "Nepal",
    "New Zealand": "New Zealand",
    "Nigeria": "Nigeria",
    "Norway": "Norway",
    "Sultanate of Oman": "Oman",
    "Palau": "Palau",
    "State of PALESTINE": "Palestine",
    "Papua New Guinea": "Papa New Guinea", # Note: Mapping to the NVIVO typo
    "Peru": "Peru",
    "The Philippines": "Philippines",
    "The State of Qatar": "Qatar",
    "The Russian Federation": "Russia",
    "Rwanda": "Rwanda",
    "Saudi Arabia": "Saudi Arabia",
    "Sierra Leone": "Sierra Leone",
    "Singapore": "Singapore",
    "Sri Lanka": "Sri Lanka",
    "Switzerland": "Switzerland",
    "Syrian Arab Republic": "Syria",
    "United Republic of Tanzania": "Tanzania",
    "Thailand": "Thailand",
    "Tonga": "Tonga",
    "TUNISIA": "Tunisia",
    "Türkiye": "Türkiye",
    "Uganda": "Uganda",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "URUGUAY": "Uruguay",
    "United States of America": "USA",
    "Yemen": "Yemen",
    "REPUBLIC OF EQUATORIAL GUINEA": None, # Explicitly drop this one

}
import unicodedata # Fixing the issue with Turkyie byte representation

def normalize_string(s):
    """A helper function to apply all cleaning steps consistently."""
    # NFC is a standard way to compose characters (e.g., u + ¨ -> ü). Take that Turkey!
    return unicodedata.normalize('NFC', s).casefold()

def align_dataframes(df_llm, df_nvivo):
    """
    Cleans, maps, filters, and sorts the DataFrames with robust string normalization.
    """
    
    # 1. Clean the NVIVO 'Country' column using the new helper function
    df_nvivo['Clean_Country'] = df_nvivo['Country'].str.replace(r'^\d+\s*:\s*', '', regex=True).str.strip().apply(normalize_string)
    
    # 2. Apply mapping and cleaning to the LLM 'Country' column
    df_llm['Clean_Country'] = df_llm['Country'].map(COUNTRY_MAPPING).fillna(df_llm['Country']).str.strip().apply(normalize_string)
    
    # 3. Filter the LLM DataFrame using the fully normalized lists
    nvivo_country_list = df_nvivo['Clean_Country'].unique()
    original_llm_count = len(df_llm)
    df_llm_aligned = df_llm[df_llm['Clean_Country'].isin(nvivo_country_list)].copy()
    filtered_count = original_llm_count - len(df_llm_aligned)
    
    if filtered_count > 0:
        print(f"✅ Filtered out {filtered_count} row(s) from the LLM data that were not in the NVIVO data.")
        dropped_countries = df_llm[~df_llm['Clean_Country'].isin(nvivo_country_list)]['Country'].tolist()
        print(f"   Dropped: {dropped_countries}")

    # 4. Sort both DataFrames to ensure row order is identical
    df_llm_aligned = df_llm_aligned.sort_values('Clean_Country').reset_index(drop=True)
    df_nvivo_aligned = df_nvivo.sort_values('Clean_Country').reset_index(drop=True)
    
    # 5. Overwrite the original 'Country' column with the clean names for consistency
    df_llm_aligned['Country'] = df_nvivo_aligned['Clean_Country']
    
    return df_llm_aligned, df_nvivo_aligned

In [181]:
df_llm_aligned, df_nvivo_aligned = align_dataframes(df_llm, df_nvivo)

# --- Final Verification ---
print("\n--- Verification ---")
print(f"NVIVO DataFrame shape after alignment: {df_nvivo_aligned.shape}")
print(f"LLM DataFrame shape after alignment: {df_llm_aligned.shape}")

are_countries_equal = df_llm_aligned['Clean_Country'].equals(df_nvivo_aligned['Clean_Country'])

if are_countries_equal:
    print("✅ SUCCESS: The 'Country' columns are now perfectly aligned and sorted.")
else:
    print("❌ FAILURE: The 'Country' columns are still not aligned. Please check the mapping.")

✅ Filtered out 1 row(s) from the LLM data that were not in the NVIVO data.
   Dropped: ['REPUBLIC OF EQUATORIAL GUINEA']

--- Verification ---
NVIVO DataFrame shape after alignment: (68, 88)
LLM DataFrame shape after alignment: (68, 88)
✅ SUCCESS: The 'Country' columns are now perfectly aligned and sorted.


In [182]:
# In[]:
import numpy as np
import pandas as pd

# --- =================================================================== ---
# --- STEP 1: PRE-PROCESSING - Binarize the NVIVO Data                  ---
# --- =================================================================== ---
code_columns_nvivo = [col for col in df_nvivo_aligned.columns if col not in ['Country', 'Clean_Country', 'join_key']]
df_nvivo_aligned[code_columns_nvivo] = (df_nvivo_aligned[code_columns_nvivo] > 0).astype(int)
print("✅ NVIVO DataFrame has been converted to binary (0s and 1s).")

# --- =================================================================== ---
# --- STEP 2: DEFINE CODES FOR ANALYSIS (Corrected)                     ---
# --- =================================================================== ---
# This list now correctly includes placeholder codes and the direct, granular codes for C9.
ANALYSIS_CODES = [
    'A : C1 Objectives - end plastic pollution', # The parent keys are only included to provide visual order in the final table
    'B : Mentioned with time frame', 'C : Mentioned, no time frame', 'D : Not mentioned', # (and to avoid having to do fancy processing)
    'E : C2 Objectives - reduce production of plastics',
    'F : Mentioned with specification', 'G : Mentioned, no specification', 'H : Not mentioned',
    'I : C3 Objectives - benefits of plastics',
    'J : Mentioned', 'K : Not mentioned',
    'L : C4 Objectives - protect human health',
    'M : Mentioned', 'N : Not mentioned',
    'O : C5 Objectives - protect biodiversity and (marine) environment',
    'P : Mentioned', 'Q : Not mentioned',
    'R : C6 Objectives - addressing the full life cycle of plastics',
    'S : Mentioned', 'T : Not mentioned', 'U : Partial mention',
    
    ('AD : 1. Upstream', ['AE : 1. Feedstock and raw materials (fossil, biobased and recycled)','AF : 2. Production and processing of monomers and polymers'], 'OR'),
    ('AG : 2. Midstream', ['AH : 1. Design phase and innovation of plastic products', 'AI : 2. Production of plastic products', 'AJ : 1. Harmful chemical additives of concern for the environment, and human health and safety (e.g., toxic to human health)', 'AK : 2. Intentionally added microplastics', 'AL : 3. Distribution', 'AM : 4. Plastics use and consumption', 'AN : Avoidable plastic - unnecessary, short lived and single use'], 'OR'),
    ('AO : 3. Downstream', ['AP : 1. Collection, sorting and waste management', 'AQ : 2. Reuse, repair and recycling', 'AR : 3. Legacy plastic'], 'OR'),
    ('AS : 4. Cross value chain', ['AT : 1. Release and emission to water, soil and air', 'AU : 2. Microplastic leakage'], 'OR'),

    # C9 Placeholders and GRANULAR Codes
    'AV : C9 Type of measure',
    'AY : Deposit systems', 'AZ : Penalty', 'BA : Public procurement', 'BB : R&D funding', 'BC : Subsidy',
    'BD : Tax incentive', 'BE : Trading system', 'BG : Ban', 'BH : EPR',
    'BI : Legal recognition - Just transition', 'BJ : Mandatory action plan', 'BK : Mandatory certification',
    'BL : Mandatory infrastructure', 'BM : Mandatory labelling', 'BN : Mandatory reports',
    'BO : Moratorium', 'BP : Performance standard', 'BQ : Requirements & surveillance in trade systems',
    'BR : Quota', 'BT : Assessment, monitoring and evaluation', 'BU : Voluntary reports',
    'BV : Education programs and awareness raising', 'BW : Expert group', 'BX : Harmonization',
    'BY : Information and guidance', 'BZ : Knowledge sharing', 'CA : Capacity building',
    'CB : Data registry', 'CC : Joint research projects', 'CD : Technology transfer',
    'CE : Promotion of research & innovation', 'CF : Voluntary certification', 'CG : Voluntary labelling',
    'CH : Target',
]


# --- =================================================================== ---
# --- STEP 3: IRR CALCULATION (Simplified Loop)                         ---
# --- =================================================================== ---
# (The Gwet's AC1 calculation functions remain the same)
def calculate_gwets_ac1(df):
    if df.shape[1] != 2 or df.shape[0] == 0: return np.nan
    subjects = df.shape[0]
    rater1_col, rater2_col = df.columns[0], df.columns[1]
    pa = (df[rater1_col] == df[rater2_col]).sum() / subjects
    pi_k = pd.melt(df)['value'].value_counts(normalize=True)
    pe = sum(p**2 for p in pi_k)
    return 1.0 if pe == 1 else (pa - pe) / (1 - pe)

def gwets_ac1_bootstrap(df, n_bootstrap=2000):
    observed_ac1 = calculate_gwets_ac1(df)
    bootstrap_ac1_scores = [calculate_gwets_ac1(df.sample(n=len(df), replace=True, random_state=i)) for i in range(n_bootstrap)]
    bootstrap_ac1_scores = [s for s in bootstrap_ac1_scores if not np.isnan(s)]
    if not bootstrap_ac1_scores: return {'ac1': observed_ac1, 'ci': (np.nan, np.nan), 'p_value': np.nan}
    lower, upper = np.percentile(bootstrap_ac1_scores, [2.5, 97.5])
    if observed_ac1 > 0: p_val = np.mean(np.array(bootstrap_ac1_scores) <= 0)
    elif observed_ac1 < 0: p_val = np.mean(np.array(bootstrap_ac1_scores) >= 0)
    else: p_val = 1.0
    p_val = min(1.0, p_val * 2)
    return {'ac1': observed_ac1, 'ci': (lower, upper), 'p_value': p_val}


results = []
df_llm_final = df_llm_aligned.set_index('Clean_Country')
df_nvivo_final = df_nvivo_aligned.set_index('Clean_Country')

print("\nCalculating IRR metrics...")
for code in ANALYSIS_CODES:
    code_name = code
    if isinstance(code, tuple):
        code_name, sub_codes, logic = code
        if logic == 'OR':
            ratings_nvivo = df_nvivo_final[sub_codes].any(axis=1).astype(int)
            ratings_llm = df_llm_final[sub_codes].any(axis=1).astype(int)
    else: # This now handles all simple and granular codes directly
        ratings_nvivo = df_nvivo_final[code_name]
        ratings_llm = df_llm_final[code_name]

    ratings_df = pd.DataFrame({'nvivo': ratings_nvivo, 'llm': ratings_llm}).dropna()
    
    if not ratings_df.empty:
        percent_agreement = (ratings_df['nvivo'] == ratings_df['llm']).mean() * 100
        pabak = (percent_agreement / 50) - 1
        ac1_result = gwets_ac1_bootstrap(ratings_df)
        results.append({ 'Codes from codebook': code_name, 'AC1 (IRR)': ac1_result['ac1'], 'Percent Agreement (%)': percent_agreement, 'PABAK': pabak, 'P-Value (Randomness)': ac1_result['p_value']})

print("✅ IRR metrics calculated.")

# --- =================================================================== ---
# --- STEP 4: RENAME CODES & DISPLAY FINAL TABLE                        ---
# --- =================================================================== ---
final_irr_table = pd.DataFrame(results)

# The rename map is now applied at the end for presentation only
rename_map = {
    'AY : Deposit systems': 'C9_Economic_Deposit_systems', 'AZ : Penalty': 'C9_Economic_Penalty',
    'BA : Public procurement': 'C9_Economic_Public_procurement', 'BB : R&D funding': 'C9_Economic_RandD_funding',
    'BC : Subsidy': 'C9_Economic_Subsidy', 'BD : Tax incentive': 'C9_Economic_Tax_incentive',
    'BE : Trading system': 'C9_Economic_Trading_system', 'BG : Ban': 'C9_Regulatory_Ban', 'BH : EPR': 'C9_Regulatory_EPR',
    'BI : Legal recognition - Just transition': 'C9_Regulatory_Legal_recognition',
    'BJ : Mandatory action plan': 'C9_Regulatory_Mandatory_action_plan',
    'BK : Mandatory certification': 'C9_Regulatory_Mandatory_certification',
    'BL : Mandatory infrastructure': 'C9_Regulatory_Mandatory_infrastructure',
    'BM : Mandatory labelling': 'C9_Regulatory_Mandatory_labelling',
    'BN : Mandatory reports': 'C9_Regulatory_Mandatory_reports', 'BO : Moratorium': 'C9_Regulatory_Moratorium',
    'BP : Performance standard': 'C9_Regulatory_Performance_standard',
    'BQ : Requirements & surveillance in trade systems': 'C9_Regulatory_Requirements_and_surveillance_in_trade_systems',
    'BT : Assessment, monitoring and evaluation': 'C9_Soft_Assessment,_monitoring_and_evaluation',
    'BV : Education programs and awareness raising': 'C9_Soft_Education_programs',
    'BW : Expert group': 'C9_Soft_Expert_group', 'BX : Harmonization': 'C9_Soft_Harmonization',
    'BY : Information and guidance': 'C9_Soft_Information_and_Guidance',
    'BZ : Knowledge sharing': 'C9_Soft_Knowledge_sharing',
    'CE : Promotion of research & innovation': 'C9_Soft_Promotion_of_RandI',
    'CF : Voluntary certification': 'C9_Soft_Voluntary_certification',
    'CG : Voluntary labelling': 'C9_Soft_Voluntary_labelling', 'CH : Target': 'C9_HasTargets'
}
#final_irr_table['Codes from codebook'] = final_irr_table['Codes from codebook'].replace(rename_map)

# Format the output table
final_irr_table["AC1 (IRR)"] = final_irr_table["AC1 (IRR)"].map('{:.3f}'.format)
final_irr_table['P-Value (Randomness)'] = final_irr_table['P-Value (Randomness)'].map(lambda p: f"{p:.3f}" if p >= 0.001 else "<0.001")
final_irr_table['Percent Agreement (%)'] = final_irr_table['Percent Agreement (%)'].map('{:.1f}'.format)
final_irr_table["PABAK"] = final_irr_table["PABAK"].map('{:.3f}'.format)

print("\n--- SCRIPT COMPLETE ---")
print("\n🎉 Final IRR Analysis Table (Replicating Original Format):")
with pd.option_context('display.max_rows', None):
    display(final_irr_table)

✅ NVIVO DataFrame has been converted to binary (0s and 1s).

Calculating IRR metrics...
✅ IRR metrics calculated.

--- SCRIPT COMPLETE ---

🎉 Final IRR Analysis Table (Replicating Original Format):


,Codes from codebook,AC1 (IRR),Percent Agreement (%),PABAK,P-Value (Randomness)
0,A : C1 Objectives - end plastic pollution,1.000,100.0,1.000,<0.001
1,B : Mentioned with time frame,0.616,91.2,0.824,0.003
2,"C : Mentioned, no time frame",0.072,63.2,0.265,0.639
3,D : Not mentioned,-0.172,70.6,0.412,<0.001
4,E : C2 Objectives - reduce production of plastics,1.000,100.0,1.000,<0.001
5,F : Mentioned with specification,0.044,82.4,0.647,0.892
6,"G : Mentioned, no specification",-0.060,47.1,-0.059,0.573
7,H : Not mentioned,-0.106,47.1,-0.059,0.332
8,I : C3 Objectives - benefits of plastics,1.000,100.0,1.000,<0.001
9,J : Mentioned,0.103,60.3,0.206,0.479


In [183]:
final_irr_table.to_csv("IRR_results.csv")